In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Configurar la conexión a la base de datos
def connect_to_db(user, password, host, database):
    """
    Establece una conexión con la base de datos usando SQLAlchemy.
    """
    connection_string = f"mysql+pymysql://{user}:{password}@{host}/{database}"
    engine = create_engine(connection_string)
    return engine

# Ejemplo de uso:
# engine = connect_to_db('root', 'password', 'localhost', 'sakila')


In [ ]:
def rentals_month(engine, month, year):
    """
    Recupera los datos de alquiler para un mes y año específicos como un DataFrame.
    """
    query = f"""
    SELECT 
        customer_id, 
        rental_id, 
        rental_date
    FROM 
        rental
    WHERE 
        MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    return pd.read_sql(query, con=engine)

# Ejemplo de uso:
# rentals_may = rentals_month(engine, 5, 2005)


In [ ]:
def rental_count_month(df, month, year):
    """
    Calcula el número de alquileres por cliente en un mes y año específicos.
    """
    rental_column = f"rentals_{month:02d}_{year}"
    rental_counts = df.groupby('customer_id').size().reset_index(name=rental_column)
    return rental_counts

# Ejemplo de uso:
# rental_counts_may = rental_count_month(rentals_may, 5, 2005)


In [ ]:
def compare_rentals(df1, df2):
    """
    Compara los alquileres entre dos DataFrames y calcula la diferencia.
    """
    combined = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    rental_columns = [col for col in combined.columns if col != 'customer_id']
    combined['difference'] = combined[rental_columns[1]] - combined[rental_columns[0]]
    return combined

# Ejemplo de uso:
# comparison = compare_rentals(rental_counts_may, rental_counts_june)


In [ ]:
# Conectar a la base de datos
engine = connect_to_db('root', 'password', 'localhost', 'sakila')

# Obtener datos de alquiler para mayo y junio de 2005
rentals_may = rentals_month(engine, 5, 2005)
rentals_june = rentals_month(engine, 6, 2005)

# Calcular el número de alquileres por cliente en mayo y junio
rental_counts_may = rental_count_month(rentals_may, 5, 2005)
rental_counts_june = rental_count_month(rentals_june, 6, 2005)

# Comparar las actividades de los clientes entre mayo y junio
comparison = compare_rentals(rental_counts_may, rental_counts_june)

# Mostrar el resultado
print(comparison)
